In [ ]:
!pip install --upgrade xlrd

     |████████████████████████████████| 96 kB 4.1 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.integrate import quad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/Data_BlackKoffer_intern/Penny Pickers_EXPE5/4.xls',sheet_name='results',header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,-0.078605,0.310538,0,0,0,0,0,1.114833,NaN,NaN,0,0
1,2,0.313415,0.310538,0,0,0,0,0,0.802123,NaN,NaN,0,0
2,3,0.015841,0.310538,0,0,0,0,0,0.571344,NaN,NaN,0,0
3,4,-0.136771,0.310538,0,0,0,0,0,0.584050,NaN,NaN,0,0
4,5,-0.263482,0.310538,0,0,0,0,0,0.583894,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,0.420082,NaN,1,0,0,0,32,0.680410,NaN,NaN,0,0
296,297,0.122331,NaN,1,0,0,0,32,0.544408,NaN,NaN,0,0
297,298,0.511536,NaN,1,0,0,0,32,0.814773,NaN,NaN,0,0
298,299,0.862973,NaN,1,0,0,0,32,0.708579,NaN,NaN,0,0


In [ ]:
def f1(sigma,t):   # t is the timestamp
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):  # we want it to go from start ... t, that's why used t+1                  
    i = k-1  # id of data point, since k is id+1 (k is in column no.0 which is id+1)
    val = math.exp(-df[1][i]*df[1][i] / (2*sigma*sigma))
    val = val / (math.sqrt(2*math.pi)*sigma)
    result = result * val
  return result

In [ ]:
def f2(t):
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):                                   # 
    i = k-1
    val = math.pi*(1 + df[1][i]*df[1][i])
    val = 1 / val
    result = result * val
  return result

In [ ]:
def m1(t):
  val = lambda t : quad(f1, 0.1,2, args=(t,))
  ans,err = val(t)
  return (1/1.9)*ans

In [ ]:
def m2(t):
  return f2(t)

In [ ]:
Pm1 = 0.5
Pm2 = 0.5

In [ ]:
def Pm1Xt(t):
  val = Pm1 * m1(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

In [ ]:
def Pm2Xt(t):
  val = Pm2 * m2(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

We win 2dollars on winning bet and loss $40 on lossing bet

In [ ]:
def sigmaapprox(t):
  start = 20*math.floor((t-1)/20)+1                                                 
  end = t
  if start==t: 
    return 0  # no variance if there is only one data point
  result = 1 / (end-start+1)
  val = 0
  for k in range(start,end+1):
    i = k-1
    val += df[1][i]*df[1][i]
  result = result * val
  result = math.sqrt(result)
  
  return result

In [ ]:
def pg(t):
  sigma = sigmaapprox(t)
  if sigma==0:   # sigma = 0 means that we do not have any data yet to calculate sigma
    return 0.0001  # very little probability of hitting target
  def integrand(x):
      result = 1 / (sigma*math.sqrt(2*math.pi))
      result = result * math.exp(-x*x / (2*sigma*sigma))
      return result
  phi,err = quad(integrand,-float('inf'),-4)
  result = 1 - 2*(phi)
  if result>=1: 
    result = 0.999   # near certain of hitting the target
  return result

In [ ]:
def pc():
  return 0.844

In [ ]:
def compute_expValue(alpha,alpha3,lamda,t):
  if t%20==0 or t%20==1: return 0   # t is the first or the last data point of a trial set
  
  Pg = pg(t)
  Pc = pc()
  a = np.exp(-(-math.log(Pg))**alpha3)
  b = lamda*np.exp(-(-math.log(1-Pg))**alpha3)
  c = np.exp(-(-math.log(Pc))**alpha3)
  d = lamda*np.exp(-(-math.log(1-Pc))**alpha3)
  V = Pm1Xt(t-1)*(a*pow(2,alpha) - b*pow(40,alpha)) + Pm2Xt(t-1)*(c*pow(2,alpha) - d*pow(40,alpha))
  return V # expected value

We bet on trial t+1 if and only if Vt > 0 

In [ ]:
def predict(alpha,alpha3,lamda,starting,ending):
  total = 0
  plist = []  # list of outcomes (bet or not bet)
  expList = []  # list of expected values for each time stamp
  for i in range(starting,ending): 
    # i is the time stamp. 
    Vt = compute_expValue(alpha,alpha3,lamda,i) # Computing expected value for timestamp i
    expList.append(Vt)
    if Vt>0:
      plist.append(1)
      if df[1][i]<=4 and df[1][i]>=-4: # in the dataframe, index of data at time i+1 is i
        total+=2 
      else: 
        total-=40
    else: plist.append(0)        # adding value for timestamp i+1
  return (plist,expList,total)       

Now we train the model (the parameter DA)

In [ ]:
Y = df[5]

In [ ]:
def errorFunc(prediction, Y):  # cost function
  total = 0
  for i in range(0,160):  
    #total += (prediction[i])**2
    total += (prediction[i]-Y[i])**2   # mean square loss
  total /= 320
  return total  

In [ ]:
def derivative_cost_function(alpha,alpha3,lamda):  # gradient of cost function (used in gradient descent)
  G_a = 0   # total gradient 
  G_a3 = 0
  G_L = 0
  
  plist,expList,total = predict(alpha,alpha3,lamda,0,160)
  
  for i in range(3,160):  
    pm1 = Pm1Xt(i-2)
    pm2 = Pm2Xt(i-2)
    Pg = pg(i-1)
    Pc = pc()
    
    d_a = (pm1*(math.exp(-(-math.log(Pg))**alpha3)*(2)**alpha*math.log(2) - lamda*math.exp(-(-math.log(1-Pg))**alpha3)*40**alpha*math.log(40)) + pm2*(math.exp(-(-math.log(Pc))**alpha3)*(2)**alpha*math.log(2) - lamda*math.exp(-(-math.log(1-Pc))**alpha3)*40**alpha*math.log(40)))/math.log(40)
    d_a3 = -(pm1*(-(-math.log(Pg))**alpha3*math.log(-math.log(Pg))*math.exp(-(-math.log(Pg))**alpha3)*(2)**alpha + lamda*(-math.log(1-Pg))**alpha3*math.log(-math.log(1-Pg))*math.exp(-(-math.log(1-Pg))**alpha3)*40**alpha) + pm2*(-(-math.log(Pc))**alpha3*math.log(-math.log(Pc))*math.exp(-(-math.log(Pc))**alpha3)*(2)**alpha + lamda*(-math.log(1-Pc))**alpha3*math.log(-math.log(1-Pc))*math.exp(-(-math.log(1-Pc))**alpha3)*40**alpha))/10
    d_L = (pm1*math.exp(-pow(-math.log(1-Pg),alpha3))*40**alpha + pm2*math.exp(-pow(-math.log(1-Pc),alpha3))*40**alpha)

    G_a += (expList[i]-Y[i])*d_a
    G_a3 += (expList[i]-Y[i])*d_a3
    G_L += (expList[i]-Y[i])*d_L
    
  
  G_a /= 160
  G_a3 /= 160
  G_L /= 160 
  
  return (G_a,G_a3,G_L)

gradient descent

In [ ]:
alpha,alpha3,lamda = 1.0,1.0,1.0

learning_rate = 0.001  # learning rate
i = 0
while True:
  if i==1000: break
  
  if i==0:
    pre_a,pre_a3,pre_L = alpha,alpha3,lamda
    i+=1
  else:
    G_a,G_a3,G_L = derivative_cost_function(alpha,alpha3,lamda)
    alpha = alpha - learning_rate*G_a
    alpha3 = alpha3 - learning_rate*G_a3
    lamda = lamda - learning_rate*G_L
  
    if alpha>1: alpha = 1.0
    if alpha3>1: alpha3 = 1.0
    if lamda<=1: lamda = 1.01
    print("alpha, alpha3, lamda: ",alpha,alpha3,lamda)
    
    if abs(alpha-pre_a)<1e-3: break
    if abs(alpha3-pre_a3)<1e-3: break
    else: pre_a,pre_a3,pre_L = alpha,alpha3,lamda
    i+=1    
if lamda<=1.01: lamda = 1.01
print("alpha, alpha3, lamda: ",alpha,alpha3,lamda)    

alpha, alpha3, lamda:  0.9858850190833386 0.998538408567553 1.0147127871149315
alpha, alpha3, lamda:  0.9728963294551113 0.9971891458092083 1.0280680363332844
alpha, alpha3, lamda:  0.960879226857387 0.995936971963778 1.0402748264870996
alpha, alpha3, lamda:  0.9497076809808072 0.9947694534002541 1.0514992218871202
alpha, alpha3, lamda:  0.939277880679358 0.9936763336463447 1.0618750093217155
alpha, alpha3, lamda:  0.9295034423292606 0.992649065958329 1.071511351105717
alpha, alpha3, lamda:  0.9203118099986985 0.991680461773478 1.0804983399493457
alpha, alpha3, lamda:  0.9203118099986985 0.991680461773478 1.0804983399493457


Tesing the resulting model on remaining 7 sessions and calculating accuracy score

In [ ]:
plist,expList,total = predict(alpha,alpha3,lamda,160,300)   # prediction

print(len(plist))
print(total)

140
94


In [ ]:
print(plist)

[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]


In [ ]:
total = 0
Y = df[5]

for i in range(0,140):  # we will have to change it to (0,160) when the dataset has a length of 320
  if (plist[i]==0 and Y[i+160]==0) or (plist[i]==1 and Y[i+160]==1): total+=1
print("The accuracy score of the base model with CbD is: ",total/140*100,"%")   # similarly here 140 -> 160

The accuracy score of the base model with CbD is:  70.71428571428572 %
